In [1]:
import ray

In [2]:
ray.shutdown()

In [3]:
ray.init()

2021-02-26 11:46:09,384	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.37',
 'raylet_ip_address': '192.168.1.37',
 'redis_address': '192.168.1.37:6379',
 'object_store_address': '/tmp/ray/session_2021-02-26_11-46-08_157773_47098/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-02-26_11-46-08_157773_47098/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-02-26_11-46-08_157773_47098',
 'metrics_export_port': 61984}

In [4]:
%config IPCompleter.use_jedi = False

In [5]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.log1p)

In [6]:
import com.ibm.research.ray.graph.Datamodel as dm

pipeline = dm.Pipeline()

node_a = dm.Node('a', transformer)
node_b = dm.Node('b', transformer)
node_c = dm.Node('c', transformer)
node_d = dm.Node('d', transformer)

pipeline.add_edge(node_a, node_b)
pipeline.add_edge(node_b, node_c)
# pipeline.add_edge(node_b, node_d)

In [7]:
X = np.array([0, 1, 2, 3])

In [8]:
transformer.transform(transformer.transform(transformer.transform(X)))

array([0.        , 0.42303586, 0.55461836, 0.62580029])

In [9]:
from com.ibm.research.ray.graph.Datamodel import XY
from com.ibm.research.ray.graph.Datamodel import XYRef

In [10]:
# Xy = XY(X, None)
# Xy_ref = ray.put(Xy)

X_ref = ray.put(X)
y_ref = ray.put(None)

Xy_ref = XYRef(X_ref, y_ref)
Xy_ref_list = [Xy_ref]
Xy_ref_ptr = ray.put(Xy_ref_list)

In [11]:
import com.ibm.research.ray.graph.Runtime as rt

In [12]:
t = [Xy_ref_ptr]

In [13]:
in_args={node_a: t}

In [14]:
from com.ibm.research.ray.graph.Runtime import ExecutionType

In [15]:
out_args = rt.execute_pipeline(pipeline, ExecutionType.TRAIN, in_args)

In [16]:
for key in out_args.keys():
    print(str(key))

c


In [17]:
ray.get(out_args[node_c])

[[<com.ibm.research.ray.graph.Datamodel.XYRef at 0x7fbc48525c50>]]

2021-02-28 11:00:54,243	ERROR worker.py:981 -- print_logs: Connection closed by server.
